# Topic Modelling

In [ ]:
import time
import pandas
#wordFreq = pandas.read_csv('WordFrequencyMatrix.csv')
#wordFreq.tail(3)

In [ ]:
articles = pandas.read_excel('articlesAnalyzed.xlsx')
articles.tail(3)

# t-SNE on Word2Vec

In [ ]:
art = [i.split() for i in articles.MWE] # list of articles
from gensim.models.word2vec import Word2Vec
from sklearn.manifold import TSNE

In [ ]:
w2v_model = Word2Vec(sentences=art,size=32, sg=1, window = 9, min_count=2,seed = 20, workers=2)

#You can save the model so you can reuse it later
w2v_model.save('word2VecModel32.w2v')

#You can reload a saved model
#w2v_model = gensim.models.Word2Vec.load('word2VecModel.w2v')
print (len(w2v_model.wv.vocab)) #words and vectors

In [ ]:
#Retrieving the vocabulary from the 32-dimensional space
X_32D=w2v_model[w2v_model.wv.vocab]
# Transform the data and load up a Panda dataframe
tSNE = TSNE(n_components=2, metric='cosine', perplexity=20, learning_rate=200, random_state=0, n_iter = 1000)
X_2D = tSNE.fit_transform(X_32D)
wordTSNE = pandas.DataFrame(X_2D, columns=['xWord','yWord'])
wordTSNE['Word'] = w2v_model.wv.vocab.keys()
wordTSNE = wordTSNE[['Word','xWord','yWord']]
wordTSNE.to_excel('word2Vec.xlsx',index = False)
wordTSNE.head(10)

In [ ]:
print('words in word2vec',len(wordTSNE.Word))

words = []
for i in articles.MWE:
    for j in i.split():
        words.append(j)
print('words in dataset',len(set(words)))
# why not the same? min_count of 2 decreasing word2Vec count?

In [ ]:
w2v_model.most_similar('mexico')

# Doc2Vec? 
# Word2Vec mapped to each document then TSNE

In [1]:
import pandas
import nltk
import numpy as np
import gensim

C:\Users\RCeballos\Anaconda\lib\site-packages\smart_open\ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')
C:\Users\RCeballos\Anaconda\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [4]:
from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors

wv = KeyedVectors.load_word2vec_format("C:\\Users\\RCeballos\\OneDrive - St. John's University\\Spring 2019\\NLP\\Project\\Project 2\\GoogleNews-vectors-negative300.bin.gz", binary=True)
# wv = KeyedVectors.load_word2vec_format("C:\\Users\\Reyes\\OneDrive - St. John's University\\Spring 2019\\NLP\\Project\\Project 2\\GoogleNews-vectors-negative300.bin.gz", binary=True)
wv.init_sims(replace=True)
# https://towardsdatascience.com/multi-class-text-classification-model-comparison-and-selection-5eb066197568
#  load a word2vec model. pre-trained by Google on a 100 billion word Google News corpus.

In [5]:
# BOW average the two word vectors

def word_averaging(wv, words):
    all_words, mean = set(), []
    
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.vocab:
            mean.append(wv.syn0norm[wv.vocab[word].index])
            all_words.add(wv.vocab[word].index)

    if not mean:
        #logging.warning("cannot compute similarity with no input %s", words)
        # FIXME: remove these examples in pre-processing
        return np.zeros(wv.vector_size,)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

def  word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, post) for post in text_list ])

In [6]:
def w2v_tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text, language='english'):
        for word in nltk.word_tokenize(sent, language='english'):
            if len(word) < 2:
                continue
            tokens.append(word)
    return tokens

In [7]:
# apply word vector averaging to tokenized text.

MWEtokenized = []
for i in articles.MWE:
    MWEtokenized.append(w2v_tokenize_text(i))

MWEtokenized_word_average = word_averaging_list(wv,MWEtokenized)

C:\Users\RCeballos\Anaconda\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `syn0norm` (Attribute will be removed in 4.0.0, use self.wv.vectors_norm instead).
  # Remove the CWD from sys.path while we load stuff.


In [8]:
len(MWEtokenized_word_average)

300

In [10]:
#Retrieving the vocabulary from the 32-dimensional space
#X_32D=w2v_model[w2v_model.wv.vocab]
# Transform the data and load up a Panda dataframe
from sklearn.manifold import TSNE

tSNE = TSNE(n_components=2, metric='cosine', perplexity=20, learning_rate=200, random_state=0, n_iter = 1000)
X_2D = tSNE.fit_transform(MWEtokenized_word_average)
wordTSNE = pandas.DataFrame(X_2D, columns=['Xdoc2vec','Ydoc2vec'])
articles = articles.join(wordTSNE)

# Topic Modeling with LDA

In [ ]:
from gensim import corpora

listMWE = [i.split() for i in articles.MWE]

dictionary = corpora.Dictionary(listMWE)
corpus = [dictionary.doc2bow(article.split()) for article in articles.MWE]
dictionary.token2id

from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim 

In [ ]:
print(time.ctime())

articlesAllTopics = {} # all possible labels
topicK = [i for i in range(2,12)] # number of topics
topicDic = {} #holds all topics and the words
models = {} #all models to later pick the best one
ModelEvaluation = [] # Perplexity /Coherence Score
#Model perplexity and topic coherence provide a convenient measure to judge how good a given topic model is. 
#topic coherence score has been more helpful.
#https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

for k in topicK:
    iterations = 100 # 20-30 good for testing, 50-100 better for final
    topic_model = gensim.models.ldamodel.LdaModel(corpus, num_topics=k, random_state=1, id2word = dictionary, passes = iterations)

    topics = topic_model.print_topics(num_words = 30)
    models['k'+str(k)] = topics # add model to dict

    for i in topics:
        topicWords = [] #all words
        for j in i[1].split(' + '):
            topicWords.append(j.split('*')[1][1:-1]) #words
        topicDic['k'+str(k)+'t'+str(i[0])] = topicWords
    print(k)

    topic = [] # main topic
    percent = [] # main topic %
    subtopic = [] # 2nd topic
    subPercent = [] # 2nd topic %

    for i in range(0,len(articles.id)):
        t = topic_model.get_document_topics(corpus[i])
        t.sort(reverse = True, key=lambda x: x[1])
        topic.append(t[0][0])
        percent.append(t[0][1])
        if len(t) > 1:
            subtopic.append(t[1][0])
            subPercent.append(t[1][1])
        else:
            subtopic.append('NA')
            subPercent.append(0)

    articlesAllTopics['Topic'+str(k)] = topic
    articlesAllTopics['Percent'+str(k)] = percent
    articlesAllTopics['Subtopic'+str(k)] = subtopic
    articlesAllTopics['SubPercent'+str(k)] = subPercent
    
    coherence_model_lda = CoherenceModel(model=topic_model, texts=listMWE, dictionary=dictionary, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    
    ModelEvaluation.append([str(k),topic_model.log_perplexity(corpus), coherence_lda]) # a measure of how good the model is. lower the better.
    
    # perplexity - a measure of how good the model is. lower the better.
    # coherence score of each topic then aggregated. the higher the better
    # https://rare-technologies.com/what-is-topic-coherence/
    
                            
    # Visualize the topics. Saves to html file
    vis = pyLDAvis.gensim.prepare(topic_model, corpus, dictionary)
    pyLDAvis.save_html(vis, 'ldaGraph'+str(k)+'.html')
    #https://pyldavis.readthedocs.io/en/latest/modules/API.html#pyLDAvis.save_html
    
print(time.ctime())

In [ ]:
#topicDic.keys()

In [ ]:
ModelEvaluation

In [ ]:
ME = {'Topic#':[],'Perplexity':[],'Coherence':[]}
for i in ModelEvaluation:
    ME['Topic#'].append(i[0])
    ME['Perplexity'].append(i[1])
    ME['Coherence'].append(i[2])
MEdf = pandas.DataFrame(ME)
MEdf.to_excel('ModelEvaluation.xlsx',index = False)
MEdf

In [ ]:
topicWords = pandas.DataFrame(topicDic )
articlesAllTopics = pandas.DataFrame(articlesAllTopics)
topicWords.tail()

In [ ]:
articlesAllTopics.tail()

In [ ]:
topicWords.to_excel('topicWords1.xlsx',index = False)
#review all words associated with each topic
articlesAllTopics.to_csv('articlesAllTopics.csv',index = False)

# Choose K
Use to visualize then choose a K

In [ ]:
import pandas
chooseTopic = pandas.read_csv('articlesAllTopics.csv')
chooseTopic.head()

In [ ]:
m = 'w'
head = True

for i in range(0,len(chooseTopic.columns),4):
    t = pandas.DataFrame({'Topic' : [chooseTopic.columns[i]]*len(chooseTopic)})
    t = t.join(chooseTopic.iloc[:,i:i+4])
    t.to_csv('chooseTopicUnpivoted.csv',mode = m, index= False, header = head)
    m = 'a'
    head = False
    print(i,'done')
# some NA's in subtopics

In [ ]:
import webbrowser
url = 'https://www.youtube.com/watch?v=VYOjWnS4cMY' #this is america
webbrowser.open(url, new=1, autoraise=True)
# notification that topic modeling is done

# Pause *** Do not proceed until you are satified with the number of K
Review all saved excel files and html files

# Merge K to Articles
After having chosen the right K

In [17]:
import pandas
chooseTopic = pandas.read_csv('articlesAllTopics.csv')
chooseTopic.tail(3)

,Topic2,Percent2,Subtopic2,SubPercent2,Topic3,Percent3,Subtopic3,SubPercent3,Topic4,Percent4,...,Subtopic9,SubPercent9,Topic10,Percent10,Subtopic10,SubPercent10,Topic11,Percent11,Subtopic11,SubPercent11
21458,0,0.936597,1,0.063403,2,0.491452,0,0.468730,0,0.750995,...,4.0,0.311107,5,0.410002,6.0,0.210001,5,0.409090,4.0,0.209091
21459,0,0.968683,1,0.031317,0,0.646079,2,0.335618,2,0.961064,...,1.0,0.160200,7,0.637699,1.0,0.133863,7,0.464668,1.0,0.156656
21460,0,0.952328,1,0.047672,2,0.711054,0,0.260141,1,0.366597,...,4.0,0.250169,0,0.686824,4.0,0.170536,0,0.481978,4.0,0.278934


In [18]:
TopicKchosen = ['Topic5', 'Percent5']
chooseTopic = chooseTopic.filter(items = TopicKchosen) # choose k columns
chooseTopic.tail(3)

,Topic5,Percent5
21458,0,0.713760
21459,2,0.687314
21460,0,0.323158


In [19]:
articles = pandas.read_excel('articlesAnalyzed.xlsx')
articles.tail(3)

,id,news_desk,pub_date,section,source,type,url,word_count,locations,subjects,...,lemma,polaritySentiWord,sentimentSentiWord,Vpolarity,Vpos,Vneg,Vsentiment,MWE,Xt_SNE,Yt_SNE
21458,5c8acca540d36ff0707a71ed,National,2018-12-31,U.S.,The New York Times,News,https://www.nytimes.com/2018/12/31/us/joyful-h...,1093,"['Hollywood (Calif)', 'Harris County (Tex)']","['Race and Ethnicity', 'Hispanic-Americans', '...",...,joyful headline race equality story race many ...,0.125,neutral,0.8591,0.373,0.000,positive,joyful headline race equality story race publ...,54.823307,39.942757
21459,5c8abf1140d36ff0707722ba,Washington,2018-12-31,U.S.,The New York Times,News,https://www.nytimes.com/2018/12/30/us/politics...,1461,[],"['Shutdowns (Institutional)', 'Border Barriers...",...,trump dig darkening hope deal end shutdown pre...,0.750,positive,0.3195,0.085,0.033,neutral,dig hope deal end shutdown president repeate...,27.929886,-38.241348
21460,5c8ae37240d36ff070800a33,U.S.,2018-12-31,U.S.,The New York Times,Interactive Feature,https://www.nytimes.com/interactive/2018/us/20...,0,[],['Two Thousand Eighteen'],...,2018 year visual story graphic selected time g...,0.625,positive,0.0000,0.000,0.000,neutral,2018 year visual story graphic selected time g...,42.510998,-52.691666


In [20]:
articles = articles.join(chooseTopic)
articles.head(2)

,id,news_desk,pub_date,section,source,type,url,word_count,locations,subjects,...,sentimentSentiWord,Vpolarity,Vpos,Vneg,Vsentiment,MWE,Xt_SNE,Yt_SNE,Topic5,Percent5
0,56869e9438f0d85a6c1302fc,U.S.,2016-01-01,U.S.,Reuters,Video,https://www.nytimes.com/video/us/1000000041215...,26,['Chicago (Ill)'],[],...,neutral,-0.8225,0.0,0.298,negative,protester denounce chicago mayor demonstrator ...,9.847380,-42.725788,1,0.812189
1,5686ced038f0d82225326f1e,Politics,2016-01-01,U.S.,The New York Times,Blog,https://www.nytimes.com/politics/first-draft/2...,374,[],"['Campaign Finance', 'Presidential Election of...",...,neutral,-0.5994,0.0,0.091,negative,hillary_clinton raised million last month hill...,-3.512983,-39.470772,0,0.352895


# Add Topic Names to File

In [31]:
articles = articles.rename(index = str, columns= {TopicKchosen[0]:'Topic',TopicKchosen[1]:'Percent'})
articles.head(1)

,ArticleID,Word,pub_date,source,type,url,word_count,headlineSnippet,polaritySentiWord,sentimentSentiWord,Vpolarity,Vpos,Vneg,Vsentiment,Xt_SNE,Yt_SNE,Topic,Percent,xWord,yWord
0,568c1b3538f0d803f4a0c4b8,000,2016-01-06,The New York Times,News,https://www.nytimes.com/2016/01/06/us/in-west-...,811,Blizzard Buried Some Dairy Cows in the Snow; 3...,-1.25,negative,-0.8126,0.022,0.148,negative,20.894001,-5.72399,2,0.451726,NaN,NaN


In [33]:
topicNames = pandas.read_excel('Topic Names.xlsx')
topicNames

,Topic,Topic Description
0,0,Election
1,1,Death/Disaster
2,2,Trump Issues
3,3,Political Issues
4,4,Scandals


In [34]:
articles = pandas.merge(articles, topicNames, how='left', on='Topic')
articles.head(1)

,ArticleID,Word,pub_date,source,type,url,word_count,headlineSnippet,polaritySentiWord,sentimentSentiWord,...,Vpos,Vneg,Vsentiment,Xt_SNE,Yt_SNE,Topic,Percent,xWord,yWord,Topic Description
0,568c1b3538f0d803f4a0c4b8,000,2016-01-06,The New York Times,News,https://www.nytimes.com/2016/01/06/us/in-west-...,811,Blizzard Buried Some Dairy Cows in the Snow; 3...,-1.25,negative,...,0.022,0.148,negative,20.894001,-5.72399,2,0.451726,NaN,NaN,Trump Issues


# Merge Articles DataFrame and wordFrequency Matrix

In [21]:
wordFreq1 = pandas.read_csv('wordFreq1.csv')
wordFreq1.tail(3)

,ArticleID,Word,Count
380599,5c8a081640d36ff0703f5b04,zinkes,0.360993
380600,5c88b59040d36ff070c72f72,zone,0.272855
380601,5c8ae47b40d36ff070803e02,zone,0.277838


In [22]:
articles.head(1)

,id,news_desk,pub_date,section,source,type,url,word_count,locations,subjects,...,sentimentSentiWord,Vpolarity,Vpos,Vneg,Vsentiment,MWE,Xt_SNE,Yt_SNE,Topic5,Percent5
0,56869e9438f0d85a6c1302fc,U.S.,2016-01-01,U.S.,Reuters,Video,https://www.nytimes.com/video/us/1000000041215...,26,['Chicago (Ill)'],[],...,neutral,-0.8225,0.0,0.298,negative,protester denounce chicago mayor demonstrator ...,9.84738,-42.725788,1,0.812189


In [23]:
articles.rename(columns = {articles.columns[0]:'ArticleID'}, inplace=True)


In [24]:
combined = pandas.merge(wordFreq1, articles, how='outer', on='ArticleID')
combined.head(1)

,ArticleID,Word,Count,news_desk,pub_date,section,source,type,url,word_count,...,sentimentSentiWord,Vpolarity,Vpos,Vneg,Vsentiment,MWE,Xt_SNE,Yt_SNE,Topic5,Percent5
0,568c1b3538f0d803f4a0c4b8,000,0.218475,National,2016-01-06,U.S.,The New York Times,News,https://www.nytimes.com/2016/01/06/us/in-west-...,811,...,negative,-0.8126,0.022,0.148,negative,"blizzard buried dairy cow snow 35,000 die 35,0...",20.894001,-5.72399,2,0.451726


In [25]:
combined.columns

Index(['ArticleID', 'Word', 'Count', 'news_desk', 'pub_date', 'section',
       'source', 'type', 'url', 'word_count', 'locations', 'subjects',
       'headline', 'snippet', 'headlineSnippet', 'tokens', 'tokensNoPunc',
       'tokensFiltered', 'postag', 'stem', 'lemma', 'polaritySentiWord',
       'sentimentSentiWord', 'Vpolarity', 'Vpos', 'Vneg', 'Vsentiment', 'MWE',
       'Xt_SNE', 'Yt_SNE', 'Topic5', 'Percent5'],
      dtype='object')

In [26]:
combined = combined.filter(items = ['ArticleID', 'Word', 'pub_date', 'source', 'type', 'url', 'word_count', 'headlineSnippet', 
                                    'Vpolarity', 'Vpos', 'Vneg', 'Vsentiment', 'Xt_SNE', 'Yt_SNE', TopicKchosen[0], TopicKchosen[1], 
                                    'Topic Description', 'Xdoc2vec','Ydoc2vec' ])

In [27]:
combined.shape

(380603, 18)

In [28]:
word2Vec = pandas.read_excel('word2Vec.xlsx')
word2Vec.head()

,Word,xWord,yWord
0,protester,26.249994,67.999649
1,denounce,9.332670,-55.858910
2,chicago,49.228859,66.641846
3,mayor,-4.235312,74.246544
4,demonstrator,26.032663,67.737495


In [29]:
combined = pandas.merge(combined, word2Vec, how='left', on='Word')
combined.head(1)

,ArticleID,Word,pub_date,source,type,url,word_count,headlineSnippet,polaritySentiWord,sentimentSentiWord,Vpolarity,Vpos,Vneg,Vsentiment,Xt_SNE,Yt_SNE,Topic5,Percent5,xWord,yWord
0,568c1b3538f0d803f4a0c4b8,000,2016-01-06,The New York Times,News,https://www.nytimes.com/2016/01/06/us/in-west-...,811,Blizzard Buried Some Dairy Cows in the Snow; 3...,-1.25,negative,-0.8126,0.022,0.148,negative,20.894001,-5.72399,2,0.451726,NaN,NaN


In [30]:
combined.shape

(380603, 20)

# Save to File Separate from Term Frequency Matrix

In [35]:
articles.to_excel('articlesTopic.xlsx',index = False)

In [36]:
combined.to_csv('NYTArticles.csv',index = False) # joined files for Tableau